<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Advanced USD

Now that we have explored the [fundamental concepts of USD](./01_USD_Fundamentals.ipynb) in the first lesson, we will learn how to use [USD's Python APIs](https://docs.omniverse.nvidia.com/py/kit/docs/api/pxr.html) to programmatically create and manipulate content.

## Setup

Just as we've mentioned in the first lesson, we wanted to highlight that visualizing USD content typically requires installing software on a workstation. For this reason, we have created specialized tools to assist in previewing content created during this lab. While these tools only surface a subset of everything USD has to offer, we hope this will provide a more wholesome learning experience, and encourage you to continue this learning journey towards [USD](https://usd.nvidia.com) and [Omniverse](https://developer.nvidia.com/nvidia-omniverse-platform).

If you are interested in the particulars of this rendering process, check out *[utils/display.py](utils/display.py)* to learn more.

In [ ]:
from utils.display import display_usd

In addition, should you want to download or edit the scripts and USD assets we'll be creating during this lesson, feel free to navigate to the */usd_files/advanced_usd* folder.

In [ ]:
base_path = "usd_files/advanced_usd/"

## Learning Objectives:

* [Opening USD Stages](#Opening-USD-Stages)
* [Composition Arcs](#USD-Format-Overview)
* [Prims, Attributes and Metadata](#Prims,-Attributes-and-Metadata)
* [Hierarchy and Traversal](#Hierarchy-and-Traversal)

## Opening USD Stages

A [stage](https://graphics.pixar.com/usd/docs/api/class_usd_stage.html#details) is the outermost container for scene description. Working with USD Stages is straight forward, as most times everything is one function call away.

In [ ]:
path = base_path + "stages/"

To load a USD file as a USD Stage we can use `Usd.Stage.Open(path)`:

In [ ]:
%%file usd_files/advanced_usd/stages/sphere_sample.usda
#usda 1.0

def Sphere "sphere"
{
}

In [ ]:
from pxr import Usd

stage = Usd.Stage.Open(path + 'sphere_sample.usda')

To create a new Stage use `Usd.Stage.CreateNew(path)`:

In [ ]:
stage = Usd.Stage.CreateNew(path + 'a_new_stage.usd')

To save a loaded Stage use `Usd.Stage.Save(path)`:

In [ ]:
stage = Usd.Stage.Open(path + 'sphere_sample.usda')
# Do something to the stage
stage.Save()  

To export a stage to a new file, we can use `Usd.Stage.Export()`. This function allows us to transition between serialization formats (*usda* or *usdc*) as well, based on the file extension provided.

In [ ]:
stage = Usd.Stage.Open(path + 'sphere_sample.usda')
# Do something to the stage
stage.Export(path + 'sphere_sample.usdc')

## Composition Arcs

The goal of this section is to introduce some basics of USD including variants, references, definitions, and schemas. 

While it does not cover the full extent of USD composition, it shows the practical foundations of some of the most useful features to get started with. 

In [ ]:
path = base_path + "comp_arcs/"

### Creating a Layer in USD

USD files represent a layer in the USD lexicon.  From the [Pixar Glossary](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Layer):
> A *Layer* is the atomic persistent container of scene description for USD.  A layer contains zero or more `PrimSpecs`, that in turn describe `Property` and `Metadata` values.  Each layer possesses an *identifier* that can be used to construct references to the layer from other layers.  Although it may be possible to someday remove this restriction, layers must currently correspond to files on a filesystem accessible via POSIX filesystem interfaces.

The following section will demonstrate how to create a basic Layer using Python.

See Pixar's [_USD Tutorials_](https://graphics.pixar.com/usd/docs/USD-Tutorials.html) for more details.

In [ ]:
from pxr import Usd, UsdGeom

# Create a temporary stage in memory
stage = Usd.Stage.CreateInMemory("SampleLayer.usda")

# Create a transform and add a sphere as mesh data
xformPrim = UsdGeom.Xform.Define(stage, "/MySphere")

# Set a translation
UsdGeom.XformCommonAPI(xformPrim).SetTranslate((7, 8, 9))

spherePrim = UsdGeom.Sphere.Define(stage, "/MySphere/MeshData")

# Get the sphere as a generic prim
sphere = stage.GetPrimAtPath("/MySphere/MeshData")

# Get the extent and radius parameters for the prim
radiusAttr = sphere.GetAttribute("radius")
extentAttr = sphere.GetAttribute("extent")

# Access the sphere schema to set the color
colorAttr = spherePrim.GetDisplayColorAttr()

# Set the radius to 2
radiusAttr.Set(2)

# Expand the extents to match the new radius
extentAttr.Set(extentAttr.Get() * 2)

# Make the sphere blue
colorAttr.Set([(0.0, 0.0, 1.0)])

# Print out the stage
print(stage.GetRootLayer().ExportToString())

# Save the resulting layer
stage.Export(path + "SampleLayer.usda", addSourceFileComment=False)

In [ ]:
display_usd(path + "SampleLayer.usda")

### Understanding how References work

One of the most basic and useful tools for composing scenes in USD is the reference. 

From the [USD Glossary](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-References): 

> The primary use for References is to compose smaller units of scene description into larger *aggregates*, building up a namespace that includes the "encapsulated" result of composing the scene description targeted by a reference.

The following example demonstrates how to make a reference, then override properties of the referenced data. 

In [ ]:
from pxr import Usd, UsdGeom

# Create a temporary stage in memory
stage = Usd.Stage.CreateInMemory('ReferenceExample.usda')

# Create a place for the reference to live
refSphere = stage.OverridePrim('/refSphere')

# Create the reference
refSphere.GetReferences().AddReference('./' + path + 'SampleLayer.usda', '/MySphere')

# Remove the translation operation applied to the base sphere's transform
# https://graphics.pixar.com/usd/release/api/class_usd_geom_xformable.html#a4b6dd6e51eb84725c763d064c4f9f3ba
refXform = UsdGeom.Xformable(refSphere)
refXform.SetXformOpOrder([])

# Print out the stage
print("The Layer\n\n")
print(stage.GetRootLayer().ExportToString())
print("\n\nThe result of Composition \n\n")
print(stage.Flatten().ExportToString())
print("\n\n")
# Override the color of the sphere to be red
overMeshData = UsdGeom.Sphere.Get(stage, '/refSphere/MeshData')
overMeshData.GetDisplayColorAttr().Set([(1,0,0)])

# Print out the stage
print("The Layer\n\n")
print(stage.GetRootLayer().ExportToString())
print("\n\nThe result of Composition \n\n")
print(stage.Flatten().ExportToString())

# Save the resulting layer
stage.GetRootLayer().Export(path + 'RefExample.usda')

### Understanding Variants

Variants and VariantSets allow a content author to provide a prim with multiple looks or forms with relative ease. 

From the [USD Glossary](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-VariantSet):
> A *VariantSet* is a composition arc that allows a content creator to package a discrete set of alternatives, between which a downstream consumer is able to non-destructively switch, or augment.  A reasonable way to think about VariantSets is as a "switchable reference".  Each Variant of a VariantSet encapsulates a tree of scene description that will be composed onto the prim on which the VariantSet is defined, when the Variant is selected.  VariantSet names must be legal USD identifiers.

In the following example we will show how to add a VariantSet that lets the content creator select different colors for their sphere.

In [ ]:
from pxr import Usd, UsdGeom

# Create a temporary  stage in memory
stage = Usd.Stage.CreateInMemory('VariantExample.usda')

# Create a place for the reference to live
variantSphere = stage.OverridePrim('/variantSphere')

# Create the reference
variantSphere.GetReferences().AddReference('./' + path + 'SampleLayer.usda', '/MySphere')

# Get the variantSphere Prim.
variantXform = stage.GetPrimAtPath('/variantSphere')

# Clear any color on the base sphere
overMeshData = UsdGeom.Sphere.Get(stage, '/variantSphere/MeshData')
colorAttr = overMeshData.GetDisplayColorAttr()
colorAttr.Clear()

# Add the VariantSet
colorVariants = variantXform.GetVariantSets().AddVariantSet('ColorsRGB')

# Add variants to the VariantSet
colorVariants.AddVariant('red')
colorVariants.AddVariant('green')
colorVariants.AddVariant('blue')

# Set the variant values
colorVariants.SetVariantSelection('red')
with colorVariants.GetVariantEditContext():
    colorAttr.Set([(1,0,0)])

colorVariants.SetVariantSelection('green')
with colorVariants.GetVariantEditContext():
    colorAttr.Set([(0,1,0)])

colorVariants.SetVariantSelection('blue')
with colorVariants.GetVariantEditContext():
    colorAttr.Set([(0,0,1)])

# Set the color to be green
colorVariants.SetVariantSelection('green')

print("\n\n The Layer\n\n")
print(stage.GetRootLayer().ExportToString())
print("\n\n The Result of Composition\n\n")
print(stage.Flatten().ExportToString())

# Save the resulting layer
stage.Export(path + 'VariantExample.usda', addSourceFileComment=False)

In [ ]:
display_usd(path + 'VariantExample.usda')

## Prims, Attributes and Metadata

Before we begin, let's define our data directory and a USD sphere.

In [ ]:
path = base_path + "prims/"

In [ ]:
%%file usd_files/advanced_usd/prims/sphere_sample.usda
#usda 1.0

def Sphere "sphere"
{
}

### Prims
Working with [Prims](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Prim) is more complicated since Prims are extremely powerful objects in USD. Prims are referenced by their path in the stage, which is a string in the form of `/Prim/ChildPrim`. `/` is a special prim known as the root prim in a stage.

To get a reference to a prim at a path use `stage_ref.GetPrimAtPath(path)`:

In [ ]:
from pxr import Sdf, Usd, UsdGeom

stage_ref = Usd.Stage.Open(path + 'sphere_sample.usda')

prim = stage_ref.GetPrimAtPath('/sphere')
print(prim.GetName()) # Prints "sphere"
print(prim.GetPrimPath()) # Prints "/sphere"

To define a new prim use `stage_ref.DefinePrim(path)`:

In [ ]:
stage_ref = Usd.Stage.Open(path + 'sphere_sample.usda')

prim = stage_ref.DefinePrim('/UnTypedPrim')
print(prim.GetName()) # Prints "UnTypedPrim"

To define a new prim with a type use `stage_ref.DefinePrim(path, type_name)` or you can use your Type's `SomeType.Define(stage_ref, path)` method:

In [ ]:
stage_ref = Usd.Stage.Open(path + 'sphere_sample.usda')

prim = stage_ref.DefinePrim('/XformPrim', 'Xform')
# Above we have a Usd.Prim, if we want to access all the Xform's types natively,
# we need to get an Xform instance of our prim
xform = UsdGeom.Xform(prim)

print(xform.GetPath()) # Prints "/XformPrim"

# It is often better to use the Define() method of your type right away, since
# it returns your typed instance rather than a Usd.Prim instance

xform_faster = UsdGeom.Xform.Define(stage_ref, '/AnotherXformPrim')

To delete a prim from the current edit layer (please refer to the [documentation about RemovePrim](https://graphics.pixar.com/usd/docs/api/class_usd_stage.html#ac605faad8fc2673263775b1eecad2955) for details), we can use `stage_ref.RemovePrim(path)`.

<b>Note:</b> the below code cell is expected to return a `expired prim` warning.

In [ ]:
stage_ref = Usd.Stage.Open(path + 'sphere_sample.usda')
prim = stage_ref.DefinePrim('/UnTypedPrim')

if stage_ref.RemovePrim('/UnTypedPrim'):
    print('/UnTypedPrim removed')

# If you try to access the prim object, it will still reference path but it is
# expired
if (prim.IsValid()):
    print('{} is valid'.format(prim.GetName()))
else:
    print('{} is not valid'.format(prim.GetName()))
  
# The above will print "Accessed invalid expired prim </UnTypedPrim>"

### Attributes
[Attributes](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Attribute) are the workhorse of storing actual data inside a Prim. Attributes are often defined as part of [Schemas](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Schema) to make it easier to access context-relevant data from within an instance of that Type.

For example, `Xform` typed Prims have an attribute called `Purpose` which is used to specify the purpose of an imageable prim. It contains one of the following values: `[default, render, proxy, guide]`

We could get this attribute's value in two ways. One, as a generic `prim_ref.GetAttribute(name)` call, but we would have to know that the exact name of the attribute we want is "purpose", and we wouldn't be able to get any code completion in an IDE that way.

The other way is to use the Xform Schema's exposed function for getting the purpose, which is `xform_ref.GetPurposeAttr()`, which returns the same object, but will be typed in an IDE and does not depend on the underlying string name of the attribute.

Most often after we get an Attribute object, we will want to get the attribute's actual value or set it. That can be done with *attribute_ref.Get()* to retrieve the value, and `attribute_ref.Set(value)` to set the value.

Let's see the code for getting an Attribute reference and getting its value:


In [ ]:
stage_ref = Usd.Stage.Open(path + 'sphere_sample.usda')

# Get a reference to the Xform instance as well as a generic Prim instance
xform_ref = UsdGeom.Xform.Define(stage_ref, '/XformPrim')
prim_ref = xform_ref.GetPrim()

# Get an attribute reference (not its value!)
purpose_from_xform_ref = xform_ref.GetPurposeAttr()
purpose_from_prim_ref = prim_ref.GetAttribute('purpose')

print(purpose_from_xform_ref == purpose_from_prim_ref) # Prints "True"

# Prints the actual attribute's value, in this case, one of [default, render,
# proxy, guide], since it is the Xform's actual Purpose attribute
print(purpose_from_xform_ref.Get())

To create an attribute that isn't part of a Type's namespace (or it is, but we want to create the attribute "manually"), we must pass the attribute name and its type to `prim_ref.CreateAttribute(name, type)`.

Otherwise, most Types expose a `Set`-style command, for example `xform_ref.SetPurposeAttr(value)`.

#### Working with Attributes

We can call the `prim_ref.CreateAttribute(name, type)` function to create the attribute, and we can use the information above to select a valid `type`. It returns a reference to the attribute created, which we can set with `attribute_ref.Set(value)`, and again we can construct a valid value by looking up the constructor above.

In [ ]:
stage_ref = Usd.Stage.CreateInMemory()

# Create a Usd.Prim
prim_ref = stage_ref.DefinePrim('/Prim')

# Create an attribute reference, using an explicit reference to the type
weight_attr = prim_ref.CreateAttribute('weight', Sdf.ValueTypeNames.Float)

print(weight_attr.Get()) # Prints empty string for default Float values, not 0!

print(weight_attr.Get() == None) # Prints "True"
print(weight_attr.Get() == 0) # Prints "False"

# To set an attribute we use the `attribute_ref.Set(value)` function
weight_attr.Set(42.3)

print(weight_attr.Get()) # Prints "42.3"

# Also, you can chain calls like so
print(prim_ref.GetPrim().GetAttribute('weight').Get()) # Prints "42.3"

## Hierarchy and Traversal

In [ ]:
path = base_path + "hierarchy/"

USD Stages are organized in a hierarchy of Prims: there is a special root prim at `/` and it may have N-number of direct Prim descendants, each of which can have their own tree of Prim descendants.

The path to a Prim is described by a string which starts with the root prim `/` and contains the Prim name separated by the path separator `/` until the last component is the desired Prim's name.

For example `/Car/Wheel/Tire` refers to the `Tire` prim which has parent `Wheel` and grandparent `Car`. `Car`'s parent is the special root prim `/`.

In the Tutorial section on Stages there is information on how to retrieve a Prim at a given path using `stage_ref.GetPrimAtPath()`.

Here is a refresher, we'll assume *car.usda* has the `/Car/Wheel/Tire` path:

In [ ]:
%%file usd_files/advanced_usd/hierarchy/car.usda
#usda 1.0

def "Car" {
    def "Wheel" {
        def "Tire" {
            
        }
    }
}

In [ ]:
from pxr import Usd

stage_ref = Usd.Stage.Open(path + 'car.usda')
prim_ref = stage_ref.GetPrimAtPath('/Car')

If we want to get a specific child of a Prim, and we know the name, we can use `prim_ref.GetChild(child_name)`:

In [ ]:
stage_ref = Usd.Stage.Open(path + 'car.usda')

prim_ref = stage_ref.GetPrimAtPath('/Car')
child_prim_ref = prim_ref.GetChild('Wheel')

# Prims can be cast as bool, so you can check if the prim exists by comparing
# its bool() overload
if child_prim_ref:
    print("/Car/Wheel exists") # This will execute

print(child_prim_ref.GetPath()) # Prints ""/Car/Wheel"

If we want to get all the children of a Prim, we can use `prim_ref.GetChildren()` which returns a list of prim references:

In [ ]:
stage_ref = Usd.Stage.Open(path + 'car.usda')

prim_ref = stage_ref.GetPrimAtPath('/Car')

# Will return [Usd.Prim(</Car/Wheel>)]
children_refs = prim_ref.GetChildren()

If we want to traverse the entire stage, the `stage_ref.Traverse()` function is perfect for that, it returns an iterator:

In [ ]:
stage_ref = Usd.Stage.Open(path + 'car.usda')

for prim_ref in stage_ref.Traverse():
    print(prim_ref.GetPath())

There are more advanced traversal methods described in the [UsdStage](https://graphics.pixar.com/usd/docs/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) documentation.

Congratulations on completing this notebook!

## Next steps

Now that you've gained some exposure to the fundamental concepts of USD and how to manipulate content using Python, we encourage you to continue on your learning journey with additional resources:
 * Refer back to the _USD Fundamentals_ to revisit some of the learnings you've made, and see how Python lets you easily apply them
 * Learn more about [the features that USD offers](https://usd.nvidia.com)
 * See how [Omniverse](https://developer.nvidia.com/nvidia-omniverse-platform) can let you express your creativity


<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>